Problem Statement: [Image Processing #1
](https://docs.google.com/document/d/1LMhYqu-Yvn38eIJV5-_1re4BOXxvcBybYgtc6ChrQ2U/edit#)

## Importing Library and Data


In [25]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import numpy as np
import pandas as pd
from sklearn import datasets

import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt 

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [26]:
import os

os.getcwd()
labels = pd.read_csv(r'/content/drive/MyDrive/Sohham/Verzeo/train.csv')
submission = pd.read_csv(r'/content/drive/MyDrive/Sohham/Verzeo/sample_submission.csv')
train_path = r'/content/drive/MyDrive/Verzeo/train/'
test_path = r'/content/drive/MyDrive/Verzeo/test/'

## Just displaying the data to get an idea

In [27]:
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [28]:
labels['has_cactus'].value_counts()

1    13136
0     4364
Name: has_cactus, dtype: int64

## Image Pre-processing

In [29]:
import numpy as np
import matplotlib.pyplot as plt

def imshow(image, ax=None, title=None, normalize=True):
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

In [30]:
class CactiDataset(Dataset):
    def __init__(self, data, path , transform = None):
        super().__init__()
        self.data = data.values
        self.path = path
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        img_name,label = self.data[index]
        img_path = os.path.join(self.path, img_name)
        image = img.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

## Normalization

In [31]:
train_transform = transforms.Compose([transforms.ToPILImage(),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

test_transform = transforms.Compose([transforms.ToPILImage(),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

valid_transform = transforms.Compose([transforms.ToPILImage(),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

## Splitting the Dataset

In [32]:
train, valid_data = train_test_split(labels, stratify=labels.has_cactus, test_size=0.2)

In [33]:
train_data = CactiDataset(train, train_path, train_transform )
valid_data = CactiDataset(valid_data, train_path, valid_transform )
test_data = CactiDataset(submission, test_path, test_transform )

In [34]:
num_epochs = 40
num_classes = 2
batch_size = 28
learning_rate = 0.001

In [35]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [36]:
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset = valid_data, batch_size = batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle=False, num_workers=0)

In [37]:
import numpy as np
import matplotlib.pyplot as plt

def imshow(image, ax=None, title=None, normalize=True):
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))
    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)
    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    return ax

## Designing a Convolution Neural Network (CNN)

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module): 
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(720, 1024)
        self.fc2 = nn.Linear(1024, 2)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [39]:
CNNmodel = CNN()
#print(CNNmodel)

## Optimization

In [40]:
CNNmodel = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

## Loss

In [41]:
train_losses = []
valid_losses = []

for epoch in range(1, num_epochs + 1):
    train_loss = 0.0
    valid_loss = 0.0
    CNNmodel.train()
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = CNNmodel(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        
    CNNmodel.eval()
    for data, target in valid_loader:
        data = data.to(device)
        target = target.to(device)
        output = CNNmodel(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
    
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
        
    print('Epoch: {} \tTraining Loss: {:f} \tValidation Loss: {:f}'.format(epoch, train_loss, valid_loss))

Epoch: 1 	Training Loss: 0.731736 	Validation Loss: 0.727068
Epoch: 2 	Training Loss: 0.732469 	Validation Loss: 0.727068
Epoch: 3 	Training Loss: 0.731606 	Validation Loss: 0.727068
Epoch: 4 	Training Loss: 0.730158 	Validation Loss: 0.727068
Epoch: 5 	Training Loss: 0.731417 	Validation Loss: 0.727068
Epoch: 6 	Training Loss: 0.731719 	Validation Loss: 0.727068
Epoch: 7 	Training Loss: 0.731849 	Validation Loss: 0.727068
Epoch: 8 	Training Loss: 0.731751 	Validation Loss: 0.727068
Epoch: 9 	Training Loss: 0.730549 	Validation Loss: 0.727068
Epoch: 10 	Training Loss: 0.731521 	Validation Loss: 0.727068
Epoch: 11 	Training Loss: 0.731990 	Validation Loss: 0.727068
Epoch: 12 	Training Loss: 0.730913 	Validation Loss: 0.727068
Epoch: 13 	Training Loss: 0.731452 	Validation Loss: 0.727068
Epoch: 14 	Training Loss: 0.730983 	Validation Loss: 0.727068
Epoch: 15 	Training Loss: 0.731615 	Validation Loss: 0.727068
Epoch: 16 	Training Loss: 0.731192 	Validation Loss: 0.727068
Epoch: 17 	Traini

Finally Testing the model on the test dataset

In [42]:
model.eval() 
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Accuracy of the model on the test dataset: {} %'.format(100 * correct/total))

torch.save(model.state_dict(), 'CNNmodel.ckpt')

Accuracy of the model on the test dataset: 99.6 %
